In [2]:
import pandas as pd

In [27]:
df_artists = pd.read_csv('artists-data.csv')
indexNames = []
for index, row in df_artists.iterrows():
    if type(row.Genres)!=str:
        indexNames.append(index)
df_artists = df_artists.drop(indexNames)
df_artists.head(10)

,Artist,Genres,Songs,Popularity,Link
0,Ivete Sangalo,Pop; Axé; Romântico,313.0,4.4,/ivete-sangalo/
1,Chiclete com Banana,Axé,268.0,3.8,/chiclete-com-banana/
2,Banda Eva,Axé; Romântico; Reggae,215.0,2.3,/banda-eva/
3,É O Tchan,Axé,129.0,1.6,/e-o-tchan/
4,Claudia Leitte,Pop; Axé; Romântico,167.0,1.5,/claudia-leitte/
5,Harmonia do Samba,Axé; Samba; Pagode,237.0,0.9,/harmonia-do-samba/
6,Ara Ketu,Axé; Pop,139.0,1.5,/ara-ketu/
7,Daniela Mercury,MPB; Axé,230.0,1.4,/daniela-mercury/
8,Olodum,Axé,74.0,1.3,/olodum/
9,Netinho,Axé,204.0,2.0,/netinho/


In [29]:
df_lyrics = pd.read_csv('lyrics-data.csv')
df_lyrics = df_lyrics.loc[lambda df: df['language'] == 'en']
df_lyrics.head(10)

,ALink,SName,SLink,Lyric,language
69,/ivete-sangalo/,Careless Whisper,/ivete-sangalo/careless-whisper.html,I feel so unsure\nAs I take your hand and lead...,en
86,/ivete-sangalo/,Could You Be Loved / Citação Musical do Rap: S...,/ivete-sangalo/could-you-be-loved-citacao-musi...,"Don't let them fool, ya\nOr even try to school...",en
88,/ivete-sangalo/,Cruisin' (Part. Saulo),/ivete-sangalo/cruisin-part-saulo.html,"Baby, let's cruise, away from here\nDon't be c...",en
111,/ivete-sangalo/,Easy,/ivete-sangalo/easy.html,"Know it sounds funny\nBut, I just can't stand ...",en
140,/ivete-sangalo/,For Your Babies (The Voice cover),/ivete-sangalo/for-your-babies-the-voice-cover...,You've got that look again\nThe one I hoped I ...,en
147,/ivete-sangalo/,Human Nature,/ivete-sangalo/human-nature.html,Looking out\nAcross the night time\nThe city w...,en
159,/ivete-sangalo/,Losing Control (Miss Cady feat. Ivete Sangalo),/ivete-sangalo/losing-control-miss-cady-feat-i...,"Uh, yeah.\nGo, go, go.\nUh, yeah.\nUh, Uh, Uhh...",en
168,/ivete-sangalo/,Master Blaster (Jammin'),/ivete-sangalo/master-blaster-jammin.html,Everyone's feeling pretty\nIt's hotter than Ju...,en
187,/ivete-sangalo/,More Than Words,/ivete-sangalo/more-than-words.html,Saying 'I Love you'\nIs not the words I want t...,en
207,/ivete-sangalo/,Natural Collie,/ivete-sangalo/natural-collie.html,Been down in the valley\nSmoking natural colli...,en


In [30]:
df_merged = pd.merge(df_lyrics, df_artists, how='inner', left_on='ALink', right_on='Link')
df_merged = df_merged.drop(['ALink','Link', "SName", "SLink", "language", "Artist", "Songs", "Popularity"], axis=1)
df_merged.head()

df_merged.to_csv('cleaned-data.csv', index = False)

In [31]:
df = pd.read_csv('cleaned-data.csv')
df.head(10)
df.tail(10)

,Lyric,Genres
191368,Ploughed the moon reached an island\nBalanced ...,World Music; Gospel/Religioso
191369,Chorus\nTake my heart away (repeat)\n\nTo be t...,World Music; Gospel/Religioso
191370,There's a brawl in a lonely whites only bar\nA...,World Music; Gospel/Religioso
191371,Through all the days that eat away\nat every b...,World Music; Gospel/Religioso
191372,Electric chair didn't burn a hair\nand the new...,World Music; Gospel/Religioso
191373,Chorus\nHere we stand waiting on the plain\nDa...,World Music; Gospel/Religioso
191374,I nearly disappeared into the mouth of a croco...,World Music; Gospel/Religioso
191375,"Amambuka, amambuka azothengisa izwe lakithi, i...",World Music; Gospel/Religioso
191376,Sweat in the heat for days on end\nwaiting for...,World Music; Gospel/Religioso
191377,Here we stand on the edge of the day\nFaces me...,World Music; Gospel/Religioso
